##### Copyright 2022 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tutorials/video/transfer_learning_with_movinet">     <img src="https://www.tensorflow.org/images/tf_logo_32px.png">     Ver en TensorFlow.org</a> </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/video/transfer_learning_with_movinet.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a> </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/video/transfer_learning_with_movinet.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tutorials/video/transfer_learning_with_movinet.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar notebook</a> </td>
</table>

# Aprendizaje por transferencia para clasificación de video con MoViNet

MoViNets (Mobile Video Networks) ofrece una familia de modelos de clasificación de video eficientes, compatibles con la inferencia en la transmisión de video. En este tutorial, se usará un modelo previamente entrenado de MoViNet para clasificar videos, específicamente para una tarea de reconocimiento de acciones, del [conjunto de datos UCF101](https://www.crcv.ucf.edu/data/UCF101.php). Un modelo previamente entrenado es una red guardada que fue entrenada antes en un conjunto de datos más grande. Para más detalles sobre MoViNets, puede consultar la publicación [MoViNets: Mobile Video Networks for Efficient Video Recognition](https://arxiv.org/abs/2103.11511) (MoViNets: redes de video móvil para reconocimiento de video eficiente) de Kondratyuk, D. et al. (2021). Con este tutorial logrará lo siguiente:

- Aprender a descargar un modelo MoViNet previamente entrenado.
- Crear un modelo nuevo usando uno previamente entrenado con un clasificador nuevo, mediante el congelamiento de la base convolucional del modelo MoViNet.
- Reemplazar la cabeza o del clasificador por el número de etiquetas de un nuevo conjunto de datos.
- Realizar el aprendizaje por transferencia en el [conjunto de datos UCF101](https://www.crcv.ucf.edu/data/UCF101.php)

El modelo descargado en este tutorial proviene de [official/projects/movinet](https://github.com/tensorflow/models/tree/master/official/projects/movinet). Este repositorio contiene una recopilación de modelos MoViNet que usa TF Hub en el formato SavedModel de TensorFlow 2.

Este tutorial de aprendizaje por transferencia es la tercera parte de una serie de tutoriales en video de TensorFlow. A continuación, compartimos otros tres tutoriales:

- [Cómo cargar datos de video](https://www.tensorflow.org/tutorials/load_data/video): en este tutorial se explica gran parte del código usado en este documento. En particular, se explica más en detalle cómo procesar y cargar los datos mediante la clase `FrameGenerator`.
- [Cómo crear un modelo 3D CNN para la clasificación de video](https://www.tensorflow.org/tutorials/video/video_classification). Tenga en cuenta que en este tutorial se usa (2+1)D CNN que descompone los aspectos espaciales y temporales de los datos en 3D. Si usa datos volumétricos como un escaneo MRI, considere utilizar un 3D CNN en vez de un (2+1)D CNN.
- [MoViNet para reconocimiento de acciones de transmisión](https://www.tensorflow.org/hub/tutorials/movinet): familiarícese con los modelos MoViNet que se encuentran disponibles en TF Hub.

## Preparar

Para descargar el modelo MoViNet previamente entrenado, empiece por instalar e importar algunas bibliotecas necesarias, incluidas: [remotezip](https://github.com/gtsystem/python-remotezip) para inspeccionar el contenido de un archivo ZIP, [tqdm](https://github.com/tqdm/tqdm) para usar la barra de progreso, [OpenCV](https://opencv.org/) para procesar archivos de video (verifique que `opencv-python` y `opencv-python-headless` sean de la misma versión) y los modelos TensorFlow ([`tf-models-official`](https://github.com/tensorflow/models/tree/master/official)). El paquete de modelos TensorFlow contiene una recopilación de modelos que se usan con las API de alto nivel de TensorFlow.

In [ ]:
!pip install remotezip tqdm opencv-python==4.5.2.52 opencv-python-headless==4.5.2.52 tf-models-official

In [ ]:
import tqdm
import random
import pathlib
import itertools
import collections

import cv2
import numpy as np
import remotezip as rz
import seaborn as sns
import matplotlib.pyplot as plt

import keras
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

# Import the MoViNet model from TensorFlow Models (tf-models-official) for the MoViNet model
from official.projects.movinet.modeling import movinet
from official.projects.movinet.modeling import movinet_model

## Carga de datos

La celda oculta que se encuentra debajo define las funciones de ayudante para descargar un slice de datos del conjunto de datos UCF-101 dataset, and load it into a `tf.data.Dataset`. En el [tutorial para carga de datos de video](https://www.tensorflow.org/tutorials/load_data/video) se brinda información detallada sobre este código.

La clase `FrameGenerator` al final del bloque oculto es la utilidad más importante en este caso. Crea un objeto iterable que puede alimentar los datos en la canalización de datos de TensorFlow. Específicamente, esta clase contiene un generador Python que carga los cuadros de video junto con su etiqueta codificada. La función del generador (`__call__`) produce el arreglo del marco emitido por `frames_from_video_file` y un vector codificado en un solo paso (one-hot) de la etiqueta asociada con el conjunto de cuadros.


In [ ]:
#@title 

def list_files_per_class(zip_url):
  """
    List the files in each class of the dataset given the zip URL.

    Args:
      zip_url: URL from which the files can be unzipped. 

    Return:
      files: List of files in each of the classes.
  """
  files = []
  with rz.RemoteZip(URL) as zip:
    for zip_info in zip.infolist():
      files.append(zip_info.filename)
  return files

def get_class(fname):
  """
    Retrieve the name of the class given a filename.

    Args:
      fname: Name of the file in the UCF101 dataset.

    Return:
      Class that the file belongs to.
  """
  return fname.split('_')[-3]

def get_files_per_class(files):
  """
    Retrieve the files that belong to each class. 

    Args:
      files: List of files in the dataset.

    Return:
      Dictionary of class names (key) and files (values).
  """
  files_for_class = collections.defaultdict(list)
  for fname in files:
    class_name = get_class(fname)
    files_for_class[class_name].append(fname)
  return files_for_class

def download_from_zip(zip_url, to_dir, file_names):
  """
    Download the contents of the zip file from the zip URL.

    Args:
      zip_url: Zip URL containing data.
      to_dir: Directory to download data to.
      file_names: Names of files to download.
  """
  with rz.RemoteZip(zip_url) as zip:
    for fn in tqdm.tqdm(file_names):
      class_name = get_class(fn)
      zip.extract(fn, str(to_dir / class_name))
      unzipped_file = to_dir / class_name / fn

      fn = pathlib.Path(fn).parts[-1]
      output_file = to_dir / class_name / fn
      unzipped_file.rename(output_file,)

def split_class_lists(files_for_class, count):
  """
    Returns the list of files belonging to a subset of data as well as the remainder of
    files that need to be downloaded.

    Args:
      files_for_class: Files belonging to a particular class of data.
      count: Number of files to download.

    Return:
      split_files: Files belonging to the subset of data.
      remainder: Dictionary of the remainder of files that need to be downloaded.
  """
  split_files = []
  remainder = {}
  for cls in files_for_class:
    split_files.extend(files_for_class[cls][:count])
    remainder[cls] = files_for_class[cls][count:]
  return split_files, remainder

def download_ufc_101_subset(zip_url, num_classes, splits, download_dir):
  """
    Download a subset of the UFC101 dataset and split them into various parts, such as
    training, validation, and test. 

    Args:
      zip_url: Zip URL containing data.
      num_classes: Number of labels.
      splits: Dictionary specifying the training, validation, test, etc. (key) division of data 
              (value is number of files per split).
      download_dir: Directory to download data to.

    Return:
      dir: Posix path of the resulting directories containing the splits of data.
  """
  files = list_files_per_class(zip_url)
  for f in files:
    tokens = f.split('/')
    if len(tokens) <= 2:
      files.remove(f) # Remove that item from the list if it does not have a filename

  files_for_class = get_files_per_class(files)

  classes = list(files_for_class.keys())[:num_classes]

  for cls in classes:
    new_files_for_class = files_for_class[cls]
    random.shuffle(new_files_for_class)
    files_for_class[cls] = new_files_for_class

  # Only use the number of classes you want in the dictionary
  files_for_class = {x: files_for_class[x] for x in list(files_for_class)[:num_classes]}

  dirs = {}
  for split_name, split_count in splits.items():
    print(split_name, ":")
    split_dir = download_dir / split_name
    split_files, files_for_class = split_class_lists(files_for_class, split_count)
    download_from_zip(zip_url, split_dir, split_files)
    dirs[split_name] = split_dir

  return dirs

def format_frames(frame, output_size):
  """
    Pad and resize an image from a video.

    Args:
      frame: Image that needs to resized and padded. 
      output_size: Pixel size of the output frame image.

    Return:
      Formatted frame with padding of specified output size.
  """
  frame = tf.image.convert_image_dtype(frame, tf.float32)
  frame = tf.image.resize_with_pad(frame, *output_size)
  return frame

def frames_from_video_file(video_path, n_frames, output_size = (224,224), frame_step = 15):
  """
    Creates frames from each video file present for each category.

    Args:
      video_path: File path to the video.
      n_frames: Number of frames to be created per video file.
      output_size: Pixel size of the output frame image.

    Return:
      An NumPy array of frames in the shape of (n_frames, height, width, channels).
  """
  # Read each video frame by frame
  result = []
  src = cv2.VideoCapture(str(video_path))  

  video_length = src.get(cv2.CAP_PROP_FRAME_COUNT)

  need_length = 1 + (n_frames - 1) * frame_step

  if need_length > video_length:
    start = 0
  else:
    max_start = video_length - need_length
    start = random.randint(0, max_start + 1)

  src.set(cv2.CAP_PROP_POS_FRAMES, start)
  # ret is a boolean indicating whether read was successful, frame is the image itself
  ret, frame = src.read()
  result.append(format_frames(frame, output_size))

  for _ in range(n_frames - 1):
    for _ in range(frame_step):
      ret, frame = src.read()
    if ret:
      frame = format_frames(frame, output_size)
      result.append(frame)
    else:
      result.append(np.zeros_like(result[0]))
  src.release()
  result = np.array(result)[..., [2, 1, 0]]

  return result

class FrameGenerator:
  def __init__(self, path, n_frames, training = False):
    """ Returns a set of frames with their associated label. 

      Args:
        path: Video file paths.
        n_frames: Number of frames. 
        training: Boolean to determine if training dataset is being created.
    """
    self.path = path
    self.n_frames = n_frames
    self.training = training
    self.class_names = sorted(set(p.name for p in self.path.iterdir() if p.is_dir()))
    self.class_ids_for_name = dict((name, idx) for idx, name in enumerate(self.class_names))

  def get_files_and_class_names(self):
    video_paths = list(self.path.glob('*/*.avi'))
    classes = [p.parent.name for p in video_paths] 
    return video_paths, classes

  def __call__(self):
    video_paths, classes = self.get_files_and_class_names()

    pairs = list(zip(video_paths, classes))

    if self.training:
      random.shuffle(pairs)

    for path, name in pairs:
      video_frames = frames_from_video_file(path, self.n_frames) 
      label = self.class_ids_for_name[name] # Encode labels
      yield video_frames, label

In [ ]:
URL = 'https://storage.googleapis.com/thumos14_files/UCF101_videos.zip'
download_dir = pathlib.Path('./UCF101_subset/')
subset_paths = download_ufc_101_subset(URL, 
                        num_classes = 10, 
                        splits = {"train": 30, "test": 20}, 
                        download_dir = download_dir)

Cree el entrenamiento y los conjuntos de datos de prueba:

In [ ]:
batch_size = 8
num_frames = 8

output_signature = (tf.TensorSpec(shape = (None, None, None, 3), dtype = tf.float32),
                    tf.TensorSpec(shape = (), dtype = tf.int16))

train_ds = tf.data.Dataset.from_generator(FrameGenerator(subset_paths['train'], num_frames, training = True),
                                          output_signature = output_signature)
train_ds = train_ds.batch(batch_size)

test_ds = tf.data.Dataset.from_generator(FrameGenerator(subset_paths['test'], num_frames),
                                         output_signature = output_signature)
test_ds = test_ds.batch(batch_size)

Las etiquetas generadas aquí representan la codificación de clases. Por ejemplo, 'ApplyEyeMakeup' está mapeada con el entero. Échele un vistazo a las etiquetas para confirmar que el conjunto de datos haya sido suficientemente aleatorizado. 

In [ ]:
for frames, labels in train_ds.take(10):
  print(labels)

Observe la forma de los datos.

In [ ]:
print(f"Shape: {frames.shape}")
print(f"Label: {labels.shape}")

## ¿Qué son los MoViNets?

Tal como lo mencionamos antes, los [MoViNets](https://arxiv.org/abs/2103.11511) son modelos de clasificación de video que se usan para transmitir videos o para inferencias en línea en tareas como la del reconocimiento de acciones. Analice la posibilidad de usar MoViNets para clasificar los datos de video para reconocimiento de acciones.

Un clasificador basado en un cuadro en 2D es eficiente y fácil de ejecutar en videos completos o para transmitir de a un cuadro por vez. Dado que no pueden tener el contexto temporal en cuenta, su exactitud es limitada y puede producir salidas inconsistentes entre un cuadro y otro.

Uno 3D CNN simple usa contexto temporal bidireccional, que puede aumentar la exactitud y la consistencia temporal. Estas redes pueden requerir de más recursos y como miran al futuro no se pueden usar para transmitir datos.

![Convolución estándar](https://www.tensorflow.org/images/tutorials/video/standard_convolution.png)

La arquitectura de MoViNet usa convoluciones en 3D que sean "causales" a lo largo del eje de tiempo (como `layers.Conv1D` con `padding="causal"`). Esta alternativa ofrece algunas de las ventajas de ambas opciones, principalmente, permite una transmisión eficiente.

![Convolución causal](https://www.tensorflow.org/images/tutorials/video/causal_convolution.png)

La convolución causal garantiza que la salida a un tiempo *t* se calcule usando solamente entradas a tiempo *t*. Para demostrar de qué manera esto puede hacer que la transmisión sea más eficiente, comience con un ejemplo más simple, con el que sienta familiaridad: una red neuronal recurrente (RNN). La RNN pasa el estado hacia adelante a través del tiempo:

![Modelo de RNN](https://www.tensorflow.org/images/tutorials/video/rnn_comparison.png)

In [ ]:
gru = layers.GRU(units=4, return_sequences=True, return_state=True)

inputs = tf.random.normal(shape=[1, 10, 8]) # (batch, sequence, channels)

result, state = gru(inputs) # Run it all at once

Al configurar el argumento `return_sequences=True` de RNN<br> se le pide que devuelva un estado al final del cálculo. Esto le permitirá pausar y después continuar donde dejó, para obtener exactamente el mismo resultado:

![Pase de estado en RNN](https://www.tensorflow.org/images/tutorials/video/rnn_state_passing.png)

In [ ]:
first_half, state = gru(inputs[:, :5, :])   # run the first half, and capture the state
second_half, _ = gru(inputs[:,5:, :], initial_state=state)  # Use the state to continue where you left off.

print(np.allclose(result[:, :5,:], first_half))
print(np.allclose(result[:, 5:,:], second_half))

Las convoluciones causales se pueden usar del mismo modo, si se tratan con cuidado y atención. Esta misma técnica se usó en [Fast Wavenet Generation Algorithm](https://arxiv.org/abs/1611.09482) (algoritmo de generación de wavenet rápido) de Le Paine et al. En la [publicación sobre MoVinet](https://arxiv.org/abs/2103.11511), el `state` se denomina "Stream Buffer" (búfer de transmisión).

![Estados pasados en una convolución causal](https://www.tensorflow.org/images/tutorials/video/causal_conv_states.png)

Al pasar esta pequeña porción del estado hacia adelante, evita recalcular el campo receptivo completo que se muestra arriba. 

## Descarga de un modelo MoViNet previamente entrenado

En esta sección, logrará lo siguiente:

1. Podrá crear un modelo MoViNet con código abierto proporcionado en [`official/projects/movinet`](https://github.com/tensorflow/models/tree/master/official/projects/movinet) de modelos TensorFlow.
2. Cargar los pesos previamente entrenados.
3. Congelar la base convolucional u otras capas excepto la cabeza del clasificador final para acelerar el ajuste fino.

Para crear el modelo, puede empezar con la configuración `a0` porque es la más rápida de entrenar en comparación con otros modelos. Consulte los [modelos MoViNet disponibles en TensorFlow Model Garden](https://github.com/tensorflow/models/blob/master/official/projects/movinet/configs/movinet.py) (el jardín modelo de TensorFlow) para hallar  algo que sea útil para su caso de uso.

In [ ]:
model_id = 'a0'
resolution = 224

tf.keras.backend.clear_session()

backbone = movinet.Movinet(model_id=model_id)
backbone.trainable = False

# Set num_classes=600 to load the pre-trained weights from the original model
model = movinet_model.MovinetClassifier(backbone=backbone, num_classes=600)
model.build([None, None, None, None, 3])

# Load pre-trained weights
!wget https://storage.googleapis.com/tf_model_garden/vision/movinet/movinet_a0_base.tar.gz -O movinet_a0_base.tar.gz -q
!tar -xvf movinet_a0_base.tar.gz

checkpoint_dir = f'movinet_{model_id}_base'
checkpoint_path = tf.train.latest_checkpoint(checkpoint_dir)
checkpoint = tf.train.Checkpoint(model=model)
status = checkpoint.restore(checkpoint_path)
status.assert_existing_objects_matched()

Para crear un clasificador, cree primero una función que contenga la columna vertebral (también conocida como extractor de atributos o backbone) y la cantidad de clases en un conjunto de datos. La función `build_classifier` tomará la columna vertebral y la cantidad de clases de un conjunto de datos para elaborar el clasificador. En este caso, el nuevo clasificador tomará salidas `num_classes` (10 clases para este subconjunto de UCF101).

In [ ]:
def build_classifier(batch_size, num_frames, resolution, backbone, num_classes):
  """Builds a classifier on top of a backbone model."""
  model = movinet_model.MovinetClassifier(
      backbone=backbone,
      num_classes=num_classes)
  model.build([batch_size, num_frames, resolution, resolution, 3])

  return model

In [ ]:
model = build_classifier(batch_size, num_frames, resolution, backbone, 10)

Para este tutorial, elija el optimizador `tf.keras.optimizers.Adam` y la función de pérdida `tf.keras.losses.SparseCategoricalCrossentropy`. Use el argumento de métricas para ver la exactitud del desempeño del modelo en cada paso.

In [ ]:
num_epochs = 2

loss_obj = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)

model.compile(loss=loss_obj, optimizer=optimizer, metrics=['accuracy'])

Entrene el modelo. Después de dos épocas, observe una pérdida baja y exactitud alta tanto para los conjuntos de entrenamiento como para los de prueba. 

In [ ]:
results = model.fit(train_ds,
                    validation_data=test_ds,
                    epochs=num_epochs,
                    validation_freq=1,
                    verbose=1)

## Evaluación del modelo

El modelo alcanzó gran exactitud en el entrenamiento del conjunto de datos. Ahora, use `Model.evaluate` de Keras para evaluarlo con el conjunto de prueba.

In [ ]:
model.evaluate(test_ds, return_dict=True)

Para visualizar mejor el desempeño del modelo, use una [matriz de confusión](https://www.tensorflow.org/api_docs/python/tf/math/confusion_matrix). La matriz de confusión le permitirá evaluar el desempeño del modelo de clasificación más allá de su exactitud. Para elaborar la matriz de confusión para este problema de clasificación multiclase, obtenga los valores reales del conjunto de prueba y los valores predichos.

In [ ]:
def get_actual_predicted_labels(dataset):
  """
    Create a list of actual ground truth values and the predictions from the model.

    Args:
      dataset: An iterable data structure, such as a TensorFlow Dataset, with features and labels.

    Return:
      Ground truth and predicted values for a particular dataset.
  """
  actual = [labels for _, labels in dataset.unbatch()]
  predicted = model.predict(dataset)

  actual = tf.stack(actual, axis=0)
  predicted = tf.concat(predicted, axis=0)
  predicted = tf.argmax(predicted, axis=1)

  return actual, predicted

In [ ]:
def plot_confusion_matrix(actual, predicted, labels, ds_type):
  cm = tf.math.confusion_matrix(actual, predicted)
  ax = sns.heatmap(cm, annot=True, fmt='g')
  sns.set(rc={'figure.figsize':(12, 12)})
  sns.set(font_scale=1.4)
  ax.set_title('Confusion matrix of action recognition for ' + ds_type)
  ax.set_xlabel('Predicted Action')
  ax.set_ylabel('Actual Action')
  plt.xticks(rotation=90)
  plt.yticks(rotation=0)
  ax.xaxis.set_ticklabels(labels)
  ax.yaxis.set_ticklabels(labels)

In [ ]:
fg = FrameGenerator(subset_paths['train'], num_frames, training = True)
label_names = list(fg.class_ids_for_name.keys())

In [ ]:
actual, predicted = get_actual_predicted_labels(test_ds)
plot_confusion_matrix(actual, predicted, label_names, 'test')

## Próximos pasos

Ahora que se ha familiarizado con el modelo MoViNet y con cómo aprovechar las distintas API de TensorFlow APIs (por ejemplo, para el aprendizaje por transferencia) intente usar el código que se muestra en este tutorial con su propio conjunto de datos. Los datos no tienen que limitarse a datos de video. Los datos volumétricos como los escaneos por MRI, también se pueden usar con las CNN 3D. Los conjuntos de datos NUSDAT y IMH mencionados en [Brain MRI-based 3D Convolutional Neural Networks for Classification of Schizophrenia and Controls](https://arxiv.org/pdf/2003.08818.pdf) (Redes neuronales convolucionales 3D basadas en MRI cerebrales para clasificación de esquizofrenia y controles) podrían ser dos fuentes de este tipo para datos de MRI.

En particular, al usar la clase `FrameGenerator` que se usó en este tutorial y los otros tutoriales sobre clasificación y datos de video le servirá de ayuda para cargar datos en sus propios modelos.

Para más información sobre cómo trabajar con datos de video en TensorFlow, consulte los siguientes tutoriales:

- [Carga de datos de video](https://www.tensorflow.org/tutorials/load_data/video)
- [Creación de un modelo de CNN 3D para clasificación de video](https://www.tensorflow.org/tutorials/video/video_classification)
- [MoViNet para reconocimiento de acciones de transmisión](https://www.tensorflow.org/hub/tutorials/movinet)